#### 1.1 Capaian Praktikum Pertemuan 5
Menganalisa dan membandingkan proses permintaan Website statis dari Web 

#### 1.2 Indikator Capaian
Aplikasi Web Server dalam menampilkan website Dinamis menggunakan metode Push, Mahasiswa dapat memahami dan mampu mengimplementasikan Web Server Python untuk web statis

#### 1.3 Landasan Teori
Pada materi pertmuan praktikum ini dijelaksan berupa Web server dalam bentuk perangkat lunak (software) Web server adalah perangkat lunak (atau gabungan perangkat keras–perangkat lunak) yang bertugas menerima permintaan (request) dari klien, biasanya browser atau aplikasi melalui protokol HTTP/HTTPS, lalu mengirimkan respon (response) berupa halaman web, data JSON, file, atau hasil pemrosesan lainnya. Server ini bersifat statis karena tidak menggunakan framework seperti Django ataupun Flask.


#### 1.4	Pelaksanaan Praktikum

##### 1.4.1	Uji Coba


##### Python Server & HTML
Pada kode dibawah ini disimpan di simpan pada sebuah direktori, apabila sudah lanjut ke tahap selanjutnya.

In [ ]:
import socket
import os
from datetime import datetime
import sys

SERVER_HOST = '127.0.0.1'
SERVER_PORT = 8080

def log_request(client_address, request_line, status_code):
    """Log incoming requests with timestamp"""
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f"[{timestamp}] {client_address[0]}:{client_address[1]} - {request_line.strip()} - Status: {status_code}")

def route_request(request):
    """Route requests to appropriate files"""
    lines = request.split('\r\n')
    request_line = lines[0]
    method, path, protocol = request_line.split()
    if path == '/':
        path = 'index.html'
    else:
        path = path.lstrip('/')
    
    return path

def get_content_type(filename):
    """Determine content type based on file extension"""
    ext = os.path.splitext(filename)[1].lower()
    content_types = {
        '.html': 'text/html',
        '.css': 'text/css',
        '.js': 'application/javascript',
        '.json': 'application/json',
        '.txt': 'text/plain',
        '.png': 'image/png',
        '.jpg': 'image/jpeg',
        '.gif': 'image/gif'
    }
    return content_types.get(ext, 'application/octet-stream')

# Create socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server_socket.bind((SERVER_HOST, SERVER_PORT))
server_socket.listen(5)
print('HTTP Server Started')
print(f'Listening on http://{SERVER_HOST}:{SERVER_PORT}')
print('Press Ctrl+C to exit\n')

request_count = 0

try:
    while True:	
        try:
            client_connection, client_address = server_socket.accept()
            request_count += 1
            
            try:
                request = client_connection.recv(1024).decode()
                
                if request:
                    filename = route_request(request)
                    if os.path.exists(filename):
                        try:
                            with open(filename, 'rb') as file:
                                body = file.read()
                            
                            response_line = 'HTTP/1.1 200 OK'.encode()
                            content_type = get_content_type(filename)
                            entity_header = f'Content-Type: {content_type}'.encode()
                            enter = '\r\n'.encode()
                            
                            response = b''.join([response_line, enter, entity_header, enter, enter, body])
                            client_connection.send(response)

                            log_request(client_address, request.split('\r\n')[0], '200 OK')
                        
                        except Exception as e:
                            error_response = b'HTTP/1.1 500 Internal Server Error\r\nContent-Type: text/html\r\n\r\n<h1>500 - Server Error</h1>'
                            client_connection.send(error_response)
                            log_request(client_address, request.split('\r\n')[0], '500 Error')
                    else:
                        not_found_html = b'HTTP/1.1 404 Not Found\r\nContent-Type: text/html\r\n\r\n<h1>404 - File Not Found</h1><p>The requested file was not found on this server.</p>'
                        client_connection.send(not_found_html)
                        log_request(client_address, request.split('\r\n')[0], '404 Not Found')
            
            except Exception as e:
                print(f"Error processing request: {e}")
            
            finally:
                client_connection.close()
        
        except KeyboardInterrupt:
            raise
        except Exception as e:
            print(f"Connection error: {e}")
            continue

except KeyboardInterrupt:
    print(f"\n\nServer Statistics:")
    print(f"Total requests handled: {request_count}")
    print(f"Server shutting down...")
    server_socket.close()
    sys.exit(0)


Kode dibawah ini merupakan HTML, dan HTML inilah berupa website yang akan dijalakan oleh Webserver Python tersebut.

In [ ]:
<!DOCTYPE html>
<html lang="en" class="h-full scroll-smooth">
  <head>
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <title>AstraWeb — Premium static site (MP5)</title>
    <meta
      name="description"
      content="A high-quality multi-page static website demo (Home, About, Services) using online assets and a modern UI."
    />
    <meta name="theme-color" content="#0b1220" />

    <link rel="preconnect" href="https://fonts.googleapis.com" />
    <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin />
    <link
      href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap"
      rel="stylesheet"
    />

    <link
      rel="icon"
      href="data:image/svg+xml,%3Csvg xmlns='http://www.w3.org/2000/svg' viewBox='0 0 64 64'%3E%3Cdefs%3E%3ClinearGradient id='g' x1='0' y1='0' x2='1' y2='1'%3E%3Cstop stop-color='%234f46e5'/%3E%3Cstop offset='1' stop-color='%230ea5e9'/%3E%3C/linearGradient%3E%3C/defs%3E%3Crect width='64' height='64' rx='16' fill='url(%23g)'/%3E%3Cpath d='M20 44 32 18 44 44h-6l-6-13-6 13z' fill='white'/%3E%3C/svg%3E"
    />

    <script>
      (() => {
        const stored = localStorage.getItem("theme");
        const prefersDark =
          window.matchMedia &&
          window.matchMedia("(prefers-color-scheme: dark)").matches;
        const theme = stored ?? (prefersDark ? "dark" : "light");
        document.documentElement.classList.toggle("dark", theme === "dark");
      })();
    </script>

    <script>
      tailwind.config = {
        darkMode: "class",
        theme: {
          extend: {
            fontFamily: {
              sans: ["Inter", "ui-sans-serif", "system-ui", "sans-serif"],
            },
          },
        },
      };
    </script>
    <script src="https://cdn.tailwindcss.com"></script>

    <style>
      summary::-webkit-details-marker {
        display: none;
      }
    </style>
  </head>

  <body
    class="min-h-full bg-white font-sans text-slate-900 antialiased selection:bg-indigo-200/70 dark:bg-slate-950 dark:text-slate-100 dark:selection:bg-indigo-500/30"
  >
    <a
      href="#content"
      class="sr-only focus:not-sr-only focus:fixed focus:left-4 focus:top-4 focus:z-50 focus:rounded-xl focus:bg-slate-900 focus:px-4 focus:py-2 focus:text-sm focus:font-semibold focus:text-white dark:focus:bg-white dark:focus:text-slate-900"
      >Skip to content</a
    >

    <header
      class="sticky top-0 z-50 border-b border-slate-200/70 bg-white/70 backdrop-blur dark:border-slate-800 dark:bg-slate-950/60"
    >
      <div class="mx-auto flex max-w-6xl items-center justify-between px-4 py-3">
        <a class="flex items-center gap-3" href="index.html">
          <span
            class="grid h-10 w-10 place-items-center rounded-2xl bg-gradient-to-br from-indigo-600 to-sky-500 text-sm font-extrabold text-white"
            >AW</span
          >
          <span class="font-semibold tracking-tight">AstraWeb</span>
        </a>

        <nav class="hidden items-center gap-6 text-sm md:flex">
          <a
            href="index.html"
            aria-current="page"
            class="font-semibold text-slate-900 dark:text-white"
            >Home</a
          >
          <a
            href="about.html"
            class="text-slate-600 hover:text-slate-900 dark:text-slate-300 dark:hover:text-white"
            >About</a
          >
          <a
            href="services.html"
            class="text-slate-600 hover:text-slate-900 dark:text-slate-300 dark:hover:text-white"
            >Services</a
          >
        </nav>

        <div class="flex items-center gap-2">
          <button
            id="themeToggle"
            type="button"
            class="inline-flex items-center justify-center rounded-xl border border-slate-200 bg-white px-3 py-2 text-sm font-semibold text-slate-700 shadow-sm hover:bg-slate-50 dark:border-slate-800 dark:bg-slate-900/60 dark:text-slate-200 dark:hover:bg-slate-900"
          >
            <span class="sr-only">Toggle theme</span>
            <svg
              class="h-5 w-5 dark:hidden"
              xmlns="http://www.w3.org/2000/svg"
              viewBox="0 0 24 24"
              fill="none"
              stroke="currentColor"
              stroke-width="2"
              stroke-linecap="round"
              stroke-linejoin="round"
            >
              <path d="M21 12.8A9 9 0 1 1 11.2 3a7 7 0 0 0 9.8 9.8Z" />
            </svg>
            <svg
              class="hidden h-5 w-5 dark:block"
              xmlns="http://www.w3.org/2000/svg"
              viewBox="0 0 24 24"
              fill="none"
              stroke="currentColor"
              stroke-width="2"
              stroke-linecap="round"
              stroke-linejoin="round"
            >
              <circle cx="12" cy="12" r="4" />
              <path d="M12 2v2M12 20v2M4.93 4.93l1.41 1.41M17.66 17.66l1.41 1.41M2 12h2M20 12h2M6.34 17.66l-1.41 1.41M19.07 4.93l-1.41 1.41" />
            </svg>
          </button>

          <a
            href="services.html#contact"
            class="hidden rounded-xl bg-slate-900 px-4 py-2 text-sm font-semibold text-white shadow-sm hover:bg-slate-800 dark:bg-white dark:text-slate-900 dark:hover:bg-slate-200 md:inline-flex"
            >Contact</a
          >

          <details class="relative md:hidden">
            <summary
              class="inline-flex cursor-pointer items-center justify-center rounded-xl border border-slate-200 bg-white px-3 py-2 text-sm font-semibold text-slate-700 shadow-sm hover:bg-slate-50 dark:border-slate-800 dark:bg-slate-900/60 dark:text-slate-200 dark:hover:bg-slate-900"
            >
              Menu
            </summary>
            <div
              class="absolute right-0 mt-2 w-56 overflow-hidden rounded-2xl border border-slate-200 bg-white shadow-lg dark:border-slate-800 dark:bg-slate-950"
            >
              <a
                class="block px-4 py-3 text-sm font-semibold text-slate-900 hover:bg-slate-50 dark:text-white dark:hover:bg-slate-900"
                href="index.html"
                >Home</a
              >
              <a
                class="block px-4 py-3 text-sm text-slate-700 hover:bg-slate-50 dark:text-slate-200 dark:hover:bg-slate-900"
                href="about.html"
                >About</a
              >
              <a
                class="block px-4 py-3 text-sm text-slate-700 hover:bg-slate-50 dark:text-slate-200 dark:hover:bg-slate-900"
                href="services.html"
                >Services</a
              >
              <div class="border-t border-slate-200 dark:border-slate-800"></div>
              <a
                class="block px-4 py-3 text-sm text-slate-700 hover:bg-slate-50 dark:text-slate-200 dark:hover:bg-slate-900"
                href="services.html#contact"
                >Contact</a
              >
            </div>
          </details>
        </div>
      </div>
    </header>

    <main id="content">
      <section class="relative overflow-hidden">
        <div
          class="absolute inset-0 -z-10 bg-gradient-to-b from-indigo-50 via-white to-white dark:from-slate-950 dark:via-slate-950 dark:to-slate-950"
        ></div>
        <div
          class="absolute inset-0 -z-10 bg-[radial-gradient(circle_at_top,rgba(79,70,229,.18),transparent_55%),radial-gradient(circle_at_bottom,rgba(14,165,233,.18),transparent_55%)] dark:bg-[radial-gradient(circle_at_top,rgba(79,70,229,.22),transparent_55%),radial-gradient(circle_at_bottom,rgba(14,165,233,.18),transparent_55%)]"
        ></div>

        <div class="mx-auto max-w-6xl px-4 py-16 md:py-24">
          <div class="grid items-center gap-12 md:grid-cols-2">
            <div>
              <div
                class="inline-flex items-center gap-2 rounded-full border border-slate-200 bg-white/70 px-3 py-1 text-xs font-semibold text-slate-700 backdrop-blur dark:border-slate-800 dark:bg-slate-950/50 dark:text-slate-200"
              >
                <span class="h-2 w-2 rounded-full bg-emerald-500"></span>
                Modern UI • Online assets • No build tools
              </div>

              <h1 class="mt-5 text-4xl font-bold tracking-tight md:text-5xl">
                A high‑quality static website,
                <span
                  class="text-transparent bg-clip-text bg-gradient-to-r from-indigo-600 to-sky-500"
                  >ready to serve</span
                >.
              </h1>

              <p class="mt-4 text-lg leading-relaxed text-slate-600 dark:text-slate-300">
                This MP5 folder now contains a clean, premium multi‑page static
                site. It’s designed to work great on your simple Python socket
                webserver and still look good when opened directly in a browser.
              </p>

              <div class="mt-8 flex flex-col gap-3 sm:flex-row">
                <a
                  href="services.html"
                  class="inline-flex items-center justify-center rounded-xl bg-slate-900 px-5 py-3 text-sm font-semibold text-white shadow-sm hover:bg-slate-800 dark:bg-white dark:text-slate-900 dark:hover:bg-slate-200"
                  >Explore services</a
                >
                <a
                  href="about.html"
                  class="inline-flex items-center justify-center rounded-xl border border-slate-200 bg-white px-5 py-3 text-sm font-semibold text-slate-900 shadow-sm hover:bg-slate-50 dark:border-slate-800 dark:bg-slate-950 dark:text-white dark:hover:bg-slate-900"
                  >Read the story</a
                >
              </div>

              <dl class="mt-10 grid grid-cols-3 gap-3">
                <div
                  class="rounded-2xl border border-slate-200 bg-white/70 p-4 backdrop-blur dark:border-slate-800 dark:bg-slate-950/50"
                >
                  <dt class="text-xs text-slate-500 dark:text-slate-400">
                    Pages
                  </dt>
                  <dd class="mt-1 text-xl font-semibold tracking-tight">3</dd>
                </div>
                <div
                  class="rounded-2xl border border-slate-200 bg-white/70 p-4 backdrop-blur dark:border-slate-800 dark:bg-slate-950/50"
                >
                  <dt class="text-xs text-slate-500 dark:text-slate-400">
                    Style
                  </dt>
                  <dd class="mt-1 text-xl font-semibold tracking-tight">
                    Modern
                  </dd>
                </div>
                <div
                  class="rounded-2xl border border-slate-200 bg-white/70 p-4 backdrop-blur dark:border-slate-800 dark:bg-slate-950/50"
                >
                  <dt class="text-xs text-slate-500 dark:text-slate-400">
                    Setup
                  </dt>
                  <dd class="mt-1 text-xl font-semibold tracking-tight">
                    1 minute
                  </dd>
                </div>
              </dl>
            </div>

            <div class="relative">
              <div
                class="overflow-hidden rounded-3xl border border-slate-200 bg-white shadow-lg dark:border-slate-800 dark:bg-slate-900"
              >
                <img
                  class="h-80 w-full object-cover md:h-[420px]"
                  src="https://images.unsplash.com/photo-1553877522-43269d4ea984?auto=format&fit=crop&w=1400&q=80"
                  alt="A tidy workspace with a laptop and notebook"
                  loading="lazy"
                />
              </div>

              <div
                class="absolute -bottom-6 -left-6 hidden w-[min(340px,90%)] rounded-3xl border border-slate-200 bg-white/80 p-5 shadow-lg backdrop-blur dark:border-slate-800 dark:bg-slate-950/60 md:block"
              >
                <p class="text-sm font-semibold">Tip</p>
                <p class="mt-1 text-sm text-slate-600 dark:text-slate-300">
                  Use the top navigation to move between pages. Try the theme
                  toggle too.
                </p>
              </div>
            </div>
          </div>
        </div>
      </section>

      <section class="mx-auto max-w-6xl px-4 py-14">
        <div class="flex items-end justify-between gap-6">
          <div>
            <h2 class="text-2xl font-bold tracking-tight">What you get</h2>
            <p class="mt-2 max-w-2xl text-slate-600 dark:text-slate-300">
              A small static site can still be polished. This set focuses on
              clean layout, nice typography, and a cohesive look.
            </p>
          </div>
          <a
            class="hidden text-sm font-semibold text-indigo-600 hover:text-indigo-500 dark:text-indigo-400 dark:hover:text-indigo-300 md:inline-flex"
            href="services.html"
            >See pricing →</a
          >
        </div>

        <div class="mt-8 grid gap-4 md:grid-cols-3">
          <div
            class="rounded-3xl border border-slate-200 bg-white p-6 shadow-sm dark:border-slate-800 dark:bg-slate-950"
          >
            <div
              class="grid h-11 w-11 place-items-center rounded-2xl bg-indigo-600/10 text-indigo-700 dark:bg-indigo-500/15 dark:text-indigo-300"
            >
              <svg
                class="h-5 w-5"
                xmlns="http://www.w3.org/2000/svg"
                viewBox="0 0 24 24"
                fill="none"
                stroke="currentColor"
                stroke-width="2"
                stroke-linecap="round"
                stroke-linejoin="round"
              >
                <path d="M12 2v20" />
                <path d="M2 12h20" />
              </svg>
            </div>
            <h3 class="mt-4 text-base font-semibold">Fast to run</h3>
            <p class="mt-2 text-sm leading-relaxed text-slate-600 dark:text-slate-300">
              Pure HTML with CDN styling—great for simple webservers and quick
              demos.
            </p>
          </div>

          <div
            class="rounded-3xl border border-slate-200 bg-white p-6 shadow-sm dark:border-slate-800 dark:bg-slate-950"
          >
            <div
              class="grid h-11 w-11 place-items-center rounded-2xl bg-sky-600/10 text-sky-700 dark:bg-sky-500/15 dark:text-sky-300"
            >
              <svg
                class="h-5 w-5"
                xmlns="http://www.w3.org/2000/svg"
                viewBox="0 0 24 24"
                fill="none"
                stroke="currentColor"
                stroke-width="2"
                stroke-linecap="round"
                stroke-linejoin="round"
              >
                <path d="M4 7h16" />
                <path d="M4 12h16" />
                <path d="M4 17h16" />
              </svg>
            </div>
            <h3 class="mt-4 text-base font-semibold">Consistent pages</h3>
            <p class="mt-2 text-sm leading-relaxed text-slate-600 dark:text-slate-300">
              Shared navigation and a design system across Home, About, and
              Services.
            </p>
          </div>

          <div
            class="rounded-3xl border border-slate-200 bg-white p-6 shadow-sm dark:border-slate-800 dark:bg-slate-950"
          >
            <div
              class="grid h-11 w-11 place-items-center rounded-2xl bg-emerald-600/10 text-emerald-700 dark:bg-emerald-500/15 dark:text-emerald-300"
            >
              <svg
                class="h-5 w-5"
                xmlns="http://www.w3.org/2000/svg"
                viewBox="0 0 24 24"
                fill="none"
                stroke="currentColor"
                stroke-width="2"
                stroke-linecap="round"
                stroke-linejoin="round"
              >
                <path d="M12 22s8-4 8-10V5l-8-3-8 3v7c0 6 8 10 8 10Z" />
                <path d="m9 12 2 2 4-4" />
              </svg>
            </div>
            <h3 class="mt-4 text-base font-semibold">Accessible basics</h3>
            <p class="mt-2 text-sm leading-relaxed text-slate-600 dark:text-slate-300">
              Semantic structure, focus styles, good contrast, and readable
              spacing.
            </p>
          </div>
        </div>
      </section>

      <section
        class="border-y border-slate-200/70 bg-slate-50 dark:border-slate-800 dark:bg-slate-950/40"
      >
        <div class="mx-auto max-w-6xl px-4 py-14">
          <div class="grid gap-6 md:grid-cols-2 md:items-center">
            <div>
              <h2 class="text-2xl font-bold tracking-tight">
                Navigate the pages
              </h2>
              <p class="mt-3 text-slate-600 dark:text-slate-300">
                Use these shortcuts to see the rest of the site.
              </p>
            </div>
            <div class="grid gap-3 sm:grid-cols-2">
              <a
                href="about.html"
                class="group rounded-3xl border border-slate-200 bg-white p-6 shadow-sm hover:bg-slate-50 dark:border-slate-800 dark:bg-slate-950 dark:hover:bg-slate-900"
              >
                <p class="text-sm font-semibold">About</p>
                <p class="mt-1 text-sm text-slate-600 dark:text-slate-300">
                  Story, values, and toolbox.
                </p>
                <p
                  class="mt-4 text-sm font-semibold text-indigo-600 group-hover:text-indigo-500 dark:text-indigo-400 dark:group-hover:text-indigo-300"
                >
                  Open →
                </p>
              </a>
              <a
                href="services.html"
                class="group rounded-3xl border border-slate-200 bg-white p-6 shadow-sm hover:bg-slate-50 dark:border-slate-800 dark:bg-slate-950 dark:hover:bg-slate-900"
              >
                <p class="text-sm font-semibold">Services</p>
                <p class="mt-1 text-sm text-slate-600 dark:text-slate-300">
                  Pricing, process, FAQ, contact.
                </p>
                <p
                  class="mt-4 text-sm font-semibold text-indigo-600 group-hover:text-indigo-500 dark:text-indigo-400 dark:group-hover:text-indigo-300"
                >
                  Open →
                </p>
              </a>
            </div>
          </div>
        </div>
      </section>

      <section class="mx-auto max-w-6xl px-4 py-16">
        <div
          class="rounded-3xl bg-slate-900 p-10 text-white shadow-lg dark:bg-white dark:text-slate-900"
        >
          <div class="grid items-center gap-8 md:grid-cols-2">
            <div>
              <h2 class="text-2xl font-bold tracking-tight">
                Ready to customize?
              </h2>
              <p class="mt-2 text-white/80 dark:text-slate-600">
                Replace the name, update the links, and swap a few images—this
                layout adapts quickly.
              </p>
            </div>
            <div class="flex flex-col gap-3 sm:flex-row md:justify-end">
              <a
                href="services.html"
                class="inline-flex items-center justify-center rounded-xl bg-white px-5 py-3 text-sm font-semibold text-slate-900 shadow-sm hover:bg-slate-100 dark:bg-slate-900 dark:text-white dark:hover:bg-slate-800"
                >View services</a
              >
              <a
                href="services.html#contact"
                class="inline-flex items-center justify-center rounded-xl border border-white/20 bg-white/10 px-5 py-3 text-sm font-semibold text-white shadow-sm hover:bg-white/15 dark:border-slate-300 dark:bg-slate-50 dark:text-slate-900 dark:hover:bg-slate-100"
                >Contact</a
              >
            </div>
          </div>
        </div>
      </section>
    </main>

    <footer class="border-t border-slate-200 py-10 dark:border-slate-800">
      <div
        class="mx-auto flex max-w-6xl flex-col gap-6 px-4 md:flex-row md:items-center md:justify-between"
      >
        <div class="flex items-center gap-3">
          <span
            class="grid h-10 w-10 place-items-center rounded-2xl bg-gradient-to-br from-indigo-600 to-sky-500 text-sm font-extrabold text-white"
            >AW</span
          >
          <div>
            <p class="text-sm font-semibold">AstraWeb</p>
            <p class="text-xs text-slate-500 dark:text-slate-400">
              MP5 static website demo
            </p>
          </div>
        </div>

        <div class="flex flex-wrap gap-x-6 gap-y-3 text-sm">
          <a
            class="text-slate-600 hover:text-slate-900 dark:text-slate-300 dark:hover:text-white"
            href="index.html"
            >Home</a
          >
          <a
            class="text-slate-600 hover:text-slate-900 dark:text-slate-300 dark:hover:text-white"
            href="about.html"
            >About</a
          >
          <a
            class="text-slate-600 hover:text-slate-900 dark:text-slate-300 dark:hover:text-white"
            href="services.html"
            >Services</a
          >
        </div>
      </div>
    </footer>

    <script>
      const toggle = document.getElementById("themeToggle");
      toggle?.addEventListener("click", () => {
        const isDark = document.documentElement.classList.toggle("dark");
        localStorage.setItem("theme", isDark ? "dark" : "light");
      });
    </script>
  </body>
</html>


File di atas disimpan dengan nama index.html pada direktori yang sama pada folder python yang anda simpan sebelumnya.

In [ ]:
<!DOCTYPE html>
<html lang="en" class="h-full scroll-smooth">
  <head>
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <title>Services — AstraWeb</title>
    <meta
      name="description"
      content="Services page with pricing tiers, process, FAQ, and a static contact form—built with online assets for a premium look."
    />
    <meta name="theme-color" content="#0b1220" />

    <link rel="preconnect" href="https://fonts.googleapis.com" />
    <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin />
    <link
      href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap"
      rel="stylesheet"
    />

    <link
      rel="icon"
      href="data:image/svg+xml,%3Csvg xmlns='http://www.w3.org/2000/svg' viewBox='0 0 64 64'%3E%3Cdefs%3E%3ClinearGradient id='g' x1='0' y1='0' x2='1' y2='1'%3E%3Cstop stop-color='%234f46e5'/%3E%3Cstop offset='1' stop-color='%230ea5e9'/%3E%3C/linearGradient%3E%3C/defs%3E%3Crect width='64' height='64' rx='16' fill='url(%23g)'/%3E%3Cpath d='M20 44 32 18 44 44h-6l-6-13-6 13z' fill='white'/%3E%3C/svg%3E"
    />

    <script>
      (() => {
        const stored = localStorage.getItem("theme");
        const prefersDark =
          window.matchMedia &&
          window.matchMedia("(prefers-color-scheme: dark)").matches;
        const theme = stored ?? (prefersDark ? "dark" : "light");
        document.documentElement.classList.toggle("dark", theme === "dark");
      })();
    </script>

    <script>
      tailwind.config = {
        darkMode: "class",
        theme: {
          extend: {
            fontFamily: {
              sans: ["Inter", "ui-sans-serif", "system-ui", "sans-serif"],
            },
          },
        },
      };
    </script>
    <script src="https://cdn.tailwindcss.com"></script>

    <style>
      summary::-webkit-details-marker {
        display: none;
      }
    </style>
  </head>

  <body
    class="min-h-full bg-white font-sans text-slate-900 antialiased selection:bg-indigo-200/70 dark:bg-slate-950 dark:text-slate-100 dark:selection:bg-indigo-500/30"
  >
    <a
      href="#content"
      class="sr-only focus:not-sr-only focus:fixed focus:left-4 focus:top-4 focus:z-50 focus:rounded-xl focus:bg-slate-900 focus:px-4 focus:py-2 focus:text-sm focus:font-semibold focus:text-white dark:focus:bg-white dark:focus:text-slate-900"
      >Skip to content</a
    >

    <header
      class="sticky top-0 z-50 border-b border-slate-200/70 bg-white/70 backdrop-blur dark:border-slate-800 dark:bg-slate-950/60"
    >
      <div class="mx-auto flex max-w-6xl items-center justify-between px-4 py-3">
        <a class="flex items-center gap-3" href="index.html">
          <span
            class="grid h-10 w-10 place-items-center rounded-2xl bg-gradient-to-br from-indigo-600 to-sky-500 text-sm font-extrabold text-white"
            >AW</span
          >
          <span class="font-semibold tracking-tight">AstraWeb</span>
        </a>

        <nav class="hidden items-center gap-6 text-sm md:flex">
          <a
            href="index.html"
            class="text-slate-600 hover:text-slate-900 dark:text-slate-300 dark:hover:text-white"
            >Home</a
          >
          <a
            href="about.html"
            class="text-slate-600 hover:text-slate-900 dark:text-slate-300 dark:hover:text-white"
            >About</a
          >
          <a
            href="services.html"
            aria-current="page"
            class="font-semibold text-slate-900 dark:text-white"
            >Services</a
          >
        </nav>

        <div class="flex items-center gap-2">
          <button
            id="themeToggle"
            type="button"
            class="inline-flex items-center justify-center rounded-xl border border-slate-200 bg-white px-3 py-2 text-sm font-semibold text-slate-700 shadow-sm hover:bg-slate-50 dark:border-slate-800 dark:bg-slate-900/60 dark:text-slate-200 dark:hover:bg-slate-900"
          >
            <span class="sr-only">Toggle theme</span>
            <svg
              class="h-5 w-5 dark:hidden"
              xmlns="http://www.w3.org/2000/svg"
              viewBox="0 0 24 24"
              fill="none"
              stroke="currentColor"
              stroke-width="2"
              stroke-linecap="round"
              stroke-linejoin="round"
            >
              <path d="M21 12.8A9 9 0 1 1 11.2 3a7 7 0 0 0 9.8 9.8Z" />
            </svg>
            <svg
              class="hidden h-5 w-5 dark:block"
              xmlns="http://www.w3.org/2000/svg"
              viewBox="0 0 24 24"
              fill="none"
              stroke="currentColor"
              stroke-width="2"
              stroke-linecap="round"
              stroke-linejoin="round"
            >
              <circle cx="12" cy="12" r="4" />
              <path d="M12 2v2M12 20v2M4.93 4.93l1.41 1.41M17.66 17.66l1.41 1.41M2 12h2M20 12h2M6.34 17.66l-1.41 1.41M19.07 4.93l-1.41 1.41" />
            </svg>
          </button>

          <a
            href="#contact"
            class="hidden rounded-xl bg-slate-900 px-4 py-2 text-sm font-semibold text-white shadow-sm hover:bg-slate-800 dark:bg-white dark:text-slate-900 dark:hover:bg-slate-200 md:inline-flex"
            >Contact</a
          >

          <details class="relative md:hidden">
            <summary
              class="inline-flex cursor-pointer items-center justify-center rounded-xl border border-slate-200 bg-white px-3 py-2 text-sm font-semibold text-slate-700 shadow-sm hover:bg-slate-50 dark:border-slate-800 dark:bg-slate-900/60 dark:text-slate-200 dark:hover:bg-slate-900"
            >
              Menu
            </summary>
            <div
              class="absolute right-0 mt-2 w-56 overflow-hidden rounded-2xl border border-slate-200 bg-white shadow-lg dark:border-slate-800 dark:bg-slate-950"
            >
              <a
                class="block px-4 py-3 text-sm text-slate-700 hover:bg-slate-50 dark:text-slate-200 dark:hover:bg-slate-900"
                href="index.html"
                >Home</a
              >
              <a
                class="block px-4 py-3 text-sm text-slate-700 hover:bg-slate-50 dark:text-slate-200 dark:hover:bg-slate-900"
                href="about.html"
                >About</a
              >
              <a
                class="block px-4 py-3 text-sm font-semibold text-slate-900 hover:bg-slate-50 dark:text-white dark:hover:bg-slate-900"
                href="services.html"
                >Services</a
              >
              <div class="border-t border-slate-200 dark:border-slate-800"></div>
              <a
                class="block px-4 py-3 text-sm text-slate-700 hover:bg-slate-50 dark:text-slate-200 dark:hover:bg-slate-900"
                href="#contact"
                >Contact</a
              >
            </div>
          </details>
        </div>
      </div>
    </header>

    <main id="content">
      <section class="relative overflow-hidden">
        <div
          class="absolute inset-0 -z-10 bg-gradient-to-b from-indigo-50 via-white to-white dark:from-slate-950 dark:via-slate-950 dark:to-slate-950"
        ></div>
        <div
          class="absolute inset-0 -z-10 bg-[radial-gradient(circle_at_top,rgba(79,70,229,.18),transparent_55%),radial-gradient(circle_at_bottom,rgba(14,165,233,.18),transparent_55%)] dark:bg-[radial-gradient(circle_at_top,rgba(79,70,229,.22),transparent_55%),radial-gradient(circle_at_bottom,rgba(14,165,233,.18),transparent_55%)]"
        ></div>

        <div class="mx-auto max-w-6xl px-4 py-16 md:py-20">
          <div class="grid items-center gap-12 md:grid-cols-2">
            <div>
              <p class="text-sm font-semibold text-indigo-600 dark:text-indigo-400">
                Services
              </p>
              <h1 class="mt-3 text-4xl font-bold tracking-tight md:text-5xl">
                A clean website kit with
                <span
                  class="text-transparent bg-clip-text bg-gradient-to-r from-indigo-600 to-sky-500"
                  >real structure</span
                >.
              </h1>
              <p class="mt-4 text-lg leading-relaxed text-slate-600 dark:text-slate-300">
                This page demonstrates a classic services layout: pricing tiers,
                delivery process, FAQ, and a contact form—all static and easy to
                customize.
              </p>

              <div class="mt-8 flex flex-col gap-3 sm:flex-row">
                <a
                  href="#pricing"
                  class="inline-flex items-center justify-center rounded-xl bg-slate-900 px-5 py-3 text-sm font-semibold text-white shadow-sm hover:bg-slate-800 dark:bg-white dark:text-slate-900 dark:hover:bg-slate-200"
                  >View pricing</a
                >
                <a
                  href="#faq"
                  class="inline-flex items-center justify-center rounded-xl border border-slate-200 bg-white px-5 py-3 text-sm font-semibold text-slate-900 shadow-sm hover:bg-slate-50 dark:border-slate-800 dark:bg-slate-950 dark:text-white dark:hover:bg-slate-900"
                  >Read FAQ</a
                >
              </div>
            </div>

            <div class="relative">
              <div
                class="overflow-hidden rounded-3xl border border-slate-200 bg-white shadow-lg dark:border-slate-800 dark:bg-slate-900"
              >
                <img
                  class="h-80 w-full object-cover md:h-[420px]"
                  src="https://images.unsplash.com/photo-1522542550221-31fd19575a2d?auto=format&fit=crop&w=1400&q=80"
                  alt="A clean desk with a notebook and coffee"
                  loading="lazy"
                />
              </div>
            </div>
          </div>
        </div>
      </section>

      <section id="pricing" class="mx-auto max-w-6xl px-4 py-14">
        <div class="flex items-end justify-between gap-6">
          <div>
            <h2 class="text-2xl font-bold tracking-tight">Pricing tiers</h2>
            <p class="mt-2 max-w-2xl text-slate-600 dark:text-slate-300">
              Example tiers you can edit for your assignment or portfolio.
            </p>
          </div>
          <a
            class="hidden text-sm font-semibold text-indigo-600 hover:text-indigo-500 dark:text-indigo-400 dark:hover:text-indigo-300 md:inline-flex"
            href="#contact"
            >Get a quote →</a
          >
        </div>

        <div class="mt-8 grid gap-4 lg:grid-cols-3">
          <article
            class="rounded-3xl border border-slate-200 bg-white p-7 shadow-sm dark:border-slate-800 dark:bg-slate-950"
          >
            <h3 class="text-sm font-semibold">Starter</h3>
            <p class="mt-2 text-3xl font-bold tracking-tight">$49</p>
            <p class="mt-2 text-sm text-slate-600 dark:text-slate-300">
              One-page landing layout with a premium look.
            </p>
            <ul class="mt-5 space-y-2 text-sm text-slate-700 dark:text-slate-200">
              <li>• Hero + features section</li>
              <li>• Mobile-first design</li>
              <li>• Basic CTA links</li>
            </ul>
          </article>

          <article
            class="rounded-3xl border border-slate-200 bg-slate-900 p-7 text-white shadow-lg dark:border-slate-800 dark:bg-white dark:text-slate-900"
          >
            <h3 class="text-sm font-semibold">Launch</h3>
            <p class="mt-2 text-3xl font-bold tracking-tight">$99</p>
            <p class="mt-2 text-sm text-white/80 dark:text-slate-600">
              A complete 3‑page setup with consistent UI.
            </p>
            <ul class="mt-5 space-y-2 text-sm text-white/90 dark:text-slate-800">
              <li>• Home / About / Services</li>
              <li>• Theme toggle + nav</li>
              <li>• Pricing, FAQ, contact section</li>
            </ul>
          </article>

          <article
            class="rounded-3xl border border-slate-200 bg-white p-7 shadow-sm dark:border-slate-800 dark:bg-slate-950"
          >
            <h3 class="text-sm font-semibold">Scale</h3>
            <p class="mt-2 text-3xl font-bold tracking-tight">$179</p>
            <p class="mt-2 text-sm text-slate-600 dark:text-slate-300">
              More sections and content polish for bigger sites.
            </p>
            <ul class="mt-5 space-y-2 text-sm text-slate-700 dark:text-slate-200">
              <li>• Extra pages or sections</li>
              <li>• Content rewrite support</li>
              <li>• Accessibility pass</li>
            </ul>
          </article>
        </div>
      </section>

      <section
        class="border-y border-slate-200/70 bg-slate-50 dark:border-slate-800 dark:bg-slate-950/40"
      >
        <div class="mx-auto max-w-6xl px-4 py-14">
          <h2 class="text-2xl font-bold tracking-tight">Process</h2>
          <p class="mt-3 max-w-3xl text-slate-600 dark:text-slate-300">
            A simple flow that maps well to static site projects.
          </p>

          <div class="mt-8 grid gap-4 md:grid-cols-3">
            <div
              class="rounded-3xl border border-slate-200 bg-white p-6 shadow-sm dark:border-slate-800 dark:bg-slate-950"
            >
              <p class="text-xs font-semibold text-slate-500 dark:text-slate-400">
                Step 01
              </p>
              <p class="mt-2 text-sm font-semibold">Plan sections</p>
              <p class="mt-2 text-sm text-slate-600 dark:text-slate-300">
                Decide what each page should contain and how they link together.
              </p>
            </div>
            <div
              class="rounded-3xl border border-slate-200 bg-white p-6 shadow-sm dark:border-slate-800 dark:bg-slate-950"
            >
              <p class="text-xs font-semibold text-slate-500 dark:text-slate-400">
                Step 02
              </p>
              <p class="mt-2 text-sm font-semibold">Build & refine</p>
              <p class="mt-2 text-sm text-slate-600 dark:text-slate-300">
                Implement components and ensure everything stays consistent.
              </p>
            </div>
            <div
              class="rounded-3xl border border-slate-200 bg-white p-6 shadow-sm dark:border-slate-800 dark:bg-slate-950"
            >
              <p class="text-xs font-semibold text-slate-500 dark:text-slate-400">
                Step 03
              </p>
              <p class="mt-2 text-sm font-semibold">Serve</p>
              <p class="mt-2 text-sm text-slate-600 dark:text-slate-300">
                Run your server in this folder and open the pages in a browser.
              </p>
            </div>
          </div>
        </div>
      </section>

      <section id="faq" class="mx-auto max-w-6xl px-4 py-14">
        <h2 class="text-2xl font-bold tracking-tight">FAQ</h2>
        <p class="mt-3 max-w-3xl text-slate-600 dark:text-slate-300">
          Quick answers about this static setup.
        </p>

        <div class="mt-8 grid gap-3">
          <details
            class="group rounded-3xl border border-slate-200 bg-white p-6 shadow-sm dark:border-slate-800 dark:bg-slate-950"
          >
            <summary class="flex cursor-pointer items-center justify-between">
              <span class="text-sm font-semibold"
                >Does the form actually send messages?</span
              >
              <span class="ml-4 text-slate-400 group-open:rotate-45" aria-hidden="true"
                >+</span
              >
            </summary>
            <p class="mt-3 text-sm text-slate-600 dark:text-slate-300">
              No—this is a static demo. The form shows an on-page confirmation.
              For real submissions you’d connect a backend or a form service.
            </p>
          </details>

          <details
            class="group rounded-3xl border border-slate-200 bg-white p-6 shadow-sm dark:border-slate-800 dark:bg-slate-950"
          >
            <summary class="flex cursor-pointer items-center justify-between">
              <span class="text-sm font-semibold"
                >Will this work on your Python socket server?</span
              >
              <span class="ml-4 text-slate-400 group-open:rotate-45" aria-hidden="true"
                >+</span
              >
            </summary>
            <p class="mt-3 text-sm text-slate-600 dark:text-slate-300">
              Yes. These pages are plain HTML and reference only CDN assets.
            </p>
          </details>

          <details
            class="group rounded-3xl border border-slate-200 bg-white p-6 shadow-sm dark:border-slate-800 dark:bg-slate-950"
          >
            <summary class="flex cursor-pointer items-center justify-between">
              <span class="text-sm font-semibold"
                >Can I open the files without a server?</span
              >
              <span class="ml-4 text-slate-400 group-open:rotate-45" aria-hidden="true"
                >+</span
              >
            </summary>
            <p class="mt-3 text-sm text-slate-600 dark:text-slate-300">
              Yes. Links use relative paths (like <code>about.html</code>) so
              local file browsing works too.
            </p>
          </details>
        </div>
      </section>

      <section id="contact" class="mx-auto max-w-6xl px-4 pb-20">
        <div class="grid gap-10 md:grid-cols-2">
          <div>
            <h2 class="text-2xl font-bold tracking-tight">Contact</h2>
            <p class="mt-3 text-slate-600 dark:text-slate-300">
              Send a message to see the static confirmation UI. Edit the text
              and fields to match your project.
            </p>

            <div
              class="mt-6 rounded-3xl bg-slate-900 p-8 text-white shadow-lg dark:bg-white dark:text-slate-900"
            >
              <p class="text-sm font-semibold">Demo contact</p>
              <p class="mt-2 text-sm text-white/80 dark:text-slate-600">
                Email: <span class="font-semibold">hello@astraweb.local</span>
              </p>
              <p class="mt-1 text-sm text-white/80 dark:text-slate-600">
                Reply time: within 1–2 days
              </p>
            </div>
          </div>

          <div
            class="rounded-3xl border border-slate-200 bg-white p-7 shadow-sm dark:border-slate-800 dark:bg-slate-950"
          >
            <form id="contactForm" class="space-y-4">
              <div>
                <label class="text-sm font-semibold" for="name">Name</label>
                <input
                  id="name"
                  name="name"
                  type="text"
                  required
                  class="mt-2 w-full rounded-2xl border border-slate-200 bg-white px-4 py-3 text-sm outline-none ring-indigo-500/30 focus:ring-4 dark:border-slate-800 dark:bg-slate-950"
                  placeholder="Your name"
                />
              </div>
              <div>
                <label class="text-sm font-semibold" for="email">Email</label>
                <input
                  id="email"
                  name="email"
                  type="email"
                  required
                  class="mt-2 w-full rounded-2xl border border-slate-200 bg-white px-4 py-3 text-sm outline-none ring-indigo-500/30 focus:ring-4 dark:border-slate-800 dark:bg-slate-950"
                  placeholder="you@example.com"
                />
              </div>
              <div>
                <label class="text-sm font-semibold" for="message">Message</label>
                <textarea
                  id="message"
                  name="message"
                  rows="4"
                  required
                  class="mt-2 w-full rounded-2xl border border-slate-200 bg-white px-4 py-3 text-sm outline-none ring-indigo-500/30 focus:ring-4 dark:border-slate-800 dark:bg-slate-950"
                  placeholder="Tell us what you want to build…"
                ></textarea>
              </div>
              <button
                type="submit"
                class="inline-flex w-full items-center justify-center rounded-xl bg-slate-900 px-5 py-3 text-sm font-semibold text-white shadow-sm hover:bg-slate-800 dark:bg-white dark:text-slate-900 dark:hover:bg-slate-200"
              >
                Send message
              </button>

              <p
                id="formNotice"
                class="hidden rounded-2xl border border-emerald-200 bg-emerald-50 px-4 py-3 text-sm text-emerald-800 dark:border-emerald-900/50 dark:bg-emerald-950/40 dark:text-emerald-200"
              >
                Message received. Thanks! (Static demo — nothing was sent.)
              </p>
            </form>
          </div>
        </div>
      </section>
    </main>

    <footer class="border-t border-slate-200 py-10 dark:border-slate-800">
      <div
        class="mx-auto flex max-w-6xl flex-col gap-6 px-4 md:flex-row md:items-center md:justify-between"
      >
        <div class="flex items-center gap-3">
          <span
            class="grid h-10 w-10 place-items-center rounded-2xl bg-gradient-to-br from-indigo-600 to-sky-500 text-sm font-extrabold text-white"
            >AW</span
          >
          <div>
            <p class="text-sm font-semibold">AstraWeb</p>
            <p class="text-xs text-slate-500 dark:text-slate-400">Services</p>
          </div>
        </div>

        <div class="flex flex-wrap gap-x-6 gap-y-3 text-sm">
          <a
            class="text-slate-600 hover:text-slate-900 dark:text-slate-300 dark:hover:text-white"
            href="index.html"
            >Home</a
          >
          <a
            class="text-slate-600 hover:text-slate-900 dark:text-slate-300 dark:hover:text-white"
            href="about.html"
            >About</a
          >
          <a
            class="text-slate-600 hover:text-slate-900 dark:text-slate-300 dark:hover:text-white"
            href="services.html"
            >Services</a
          >
          <a
            class="text-slate-600 hover:text-slate-900 dark:text-slate-300 dark:hover:text-white"
            href="#contact"
            >Contact</a
          >
        </div>
      </div>
    </footer>

    <script>
      const toggle = document.getElementById("themeToggle");
      toggle?.addEventListener("click", () => {
        const isDark = document.documentElement.classList.toggle("dark");
        localStorage.setItem("theme", isDark ? "dark" : "light");
      });

      const form = document.getElementById("contactForm");
      const notice = document.getElementById("formNotice");

      form?.addEventListener("submit", (event) => {
        event.preventDefault();
        form.reset();
        notice?.classList.remove("hidden");
        notice?.scrollIntoView({ behavior: "smooth", block: "nearest" });
        window.setTimeout(() => notice?.classList.add("hidden"), 5000);
      });
    </script>
  </body>
</html>


File di atas disimpan dengan nama services.html pada direktori yang sama pada folder python yang anda simpan sebelumnya.

In [ ]:
<!DOCTYPE html>
<html lang="en" class="h-full scroll-smooth">
  <head>
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <title>About — AstraWeb</title>
    <meta
      name="description"
      content="About AstraWeb: a modern multi-page static site demo with a consistent design system and online assets."
    />
    <meta name="theme-color" content="#0b1220" />

    <link rel="preconnect" href="https://fonts.googleapis.com" />
    <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin />
    <link
      href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap"
      rel="stylesheet"
    />

    <link
      rel="icon"
      href="data:image/svg+xml,%3Csvg xmlns='http://www.w3.org/2000/svg' viewBox='0 0 64 64'%3E%3Cdefs%3E%3ClinearGradient id='g' x1='0' y1='0' x2='1' y2='1'%3E%3Cstop stop-color='%234f46e5'/%3E%3Cstop offset='1' stop-color='%230ea5e9'/%3E%3C/linearGradient%3E%3C/defs%3E%3Crect width='64' height='64' rx='16' fill='url(%23g)'/%3E%3Cpath d='M20 44 32 18 44 44h-6l-6-13-6 13z' fill='white'/%3E%3C/svg%3E"
    />

    <script>
      (() => {
        const stored = localStorage.getItem("theme");
        const prefersDark =
          window.matchMedia &&
          window.matchMedia("(prefers-color-scheme: dark)").matches;
        const theme = stored ?? (prefersDark ? "dark" : "light");
        document.documentElement.classList.toggle("dark", theme === "dark");
      })();
    </script>

    <script>
      tailwind.config = {
        darkMode: "class",
        theme: {
          extend: {
            fontFamily: {
              sans: ["Inter", "ui-sans-serif", "system-ui", "sans-serif"],
            },
          },
        },
      };
    </script>
    <script src="https://cdn.tailwindcss.com"></script>

    <style>
      summary::-webkit-details-marker {
        display: none;
      }
    </style>
  </head>

  <body
    class="min-h-full bg-white font-sans text-slate-900 antialiased selection:bg-indigo-200/70 dark:bg-slate-950 dark:text-slate-100 dark:selection:bg-indigo-500/30"
  >
    <a
      href="#content"
      class="sr-only focus:not-sr-only focus:fixed focus:left-4 focus:top-4 focus:z-50 focus:rounded-xl focus:bg-slate-900 focus:px-4 focus:py-2 focus:text-sm focus:font-semibold focus:text-white dark:focus:bg-white dark:focus:text-slate-900"
      >Skip to content</a
    >

    <header
      class="sticky top-0 z-50 border-b border-slate-200/70 bg-white/70 backdrop-blur dark:border-slate-800 dark:bg-slate-950/60"
    >
      <div class="mx-auto flex max-w-6xl items-center justify-between px-4 py-3">
        <a class="flex items-center gap-3" href="index.html">
          <span
            class="grid h-10 w-10 place-items-center rounded-2xl bg-gradient-to-br from-indigo-600 to-sky-500 text-sm font-extrabold text-white"
            >AW</span
          >
          <span class="font-semibold tracking-tight">AstraWeb</span>
        </a>

        <nav class="hidden items-center gap-6 text-sm md:flex">
          <a
            href="index.html"
            class="text-slate-600 hover:text-slate-900 dark:text-slate-300 dark:hover:text-white"
            >Home</a
          >
          <a
            href="about.html"
            aria-current="page"
            class="font-semibold text-slate-900 dark:text-white"
            >About</a
          >
          <a
            href="services.html"
            class="text-slate-600 hover:text-slate-900 dark:text-slate-300 dark:hover:text-white"
            >Services</a
          >
        </nav>

        <div class="flex items-center gap-2">
          <button
            id="themeToggle"
            type="button"
            class="inline-flex items-center justify-center rounded-xl border border-slate-200 bg-white px-3 py-2 text-sm font-semibold text-slate-700 shadow-sm hover:bg-slate-50 dark:border-slate-800 dark:bg-slate-900/60 dark:text-slate-200 dark:hover:bg-slate-900"
          >
            <span class="sr-only">Toggle theme</span>
            <svg
              class="h-5 w-5 dark:hidden"
              xmlns="http://www.w3.org/2000/svg"
              viewBox="0 0 24 24"
              fill="none"
              stroke="currentColor"
              stroke-width="2"
              stroke-linecap="round"
              stroke-linejoin="round"
            >
              <path d="M21 12.8A9 9 0 1 1 11.2 3a7 7 0 0 0 9.8 9.8Z" />
            </svg>
            <svg
              class="hidden h-5 w-5 dark:block"
              xmlns="http://www.w3.org/2000/svg"
              viewBox="0 0 24 24"
              fill="none"
              stroke="currentColor"
              stroke-width="2"
              stroke-linecap="round"
              stroke-linejoin="round"
            >
              <circle cx="12" cy="12" r="4" />
              <path d="M12 2v2M12 20v2M4.93 4.93l1.41 1.41M17.66 17.66l1.41 1.41M2 12h2M20 12h2M6.34 17.66l-1.41 1.41M19.07 4.93l-1.41 1.41" />
            </svg>
          </button>

          <a
            href="services.html#contact"
            class="hidden rounded-xl bg-slate-900 px-4 py-2 text-sm font-semibold text-white shadow-sm hover:bg-slate-800 dark:bg-white dark:text-slate-900 dark:hover:bg-slate-200 md:inline-flex"
            >Contact</a
          >

          <details class="relative md:hidden">
            <summary
              class="inline-flex cursor-pointer items-center justify-center rounded-xl border border-slate-200 bg-white px-3 py-2 text-sm font-semibold text-slate-700 shadow-sm hover:bg-slate-50 dark:border-slate-800 dark:bg-slate-900/60 dark:text-slate-200 dark:hover:bg-slate-900"
            >
              Menu
            </summary>
            <div
              class="absolute right-0 mt-2 w-56 overflow-hidden rounded-2xl border border-slate-200 bg-white shadow-lg dark:border-slate-800 dark:bg-slate-950"
            >
              <a
                class="block px-4 py-3 text-sm text-slate-700 hover:bg-slate-50 dark:text-slate-200 dark:hover:bg-slate-900"
                href="index.html"
                >Home</a
              >
              <a
                class="block px-4 py-3 text-sm font-semibold text-slate-900 hover:bg-slate-50 dark:text-white dark:hover:bg-slate-900"
                href="about.html"
                >About</a
              >
              <a
                class="block px-4 py-3 text-sm text-slate-700 hover:bg-slate-50 dark:text-slate-200 dark:hover:bg-slate-900"
                href="services.html"
                >Services</a
              >
              <div class="border-t border-slate-200 dark:border-slate-800"></div>
              <a
                class="block px-4 py-3 text-sm text-slate-700 hover:bg-slate-50 dark:text-slate-200 dark:hover:bg-slate-900"
                href="services.html#contact"
                >Contact</a
              >
            </div>
          </details>
        </div>
      </div>
    </header>

    <main id="content">
      <section class="relative overflow-hidden">
        <div
          class="absolute inset-0 -z-10 bg-gradient-to-b from-sky-50 via-white to-white dark:from-slate-950 dark:via-slate-950 dark:to-slate-950"
        ></div>
        <div
          class="absolute inset-0 -z-10 bg-[radial-gradient(circle_at_top,rgba(14,165,233,.18),transparent_55%),radial-gradient(circle_at_bottom,rgba(79,70,229,.16),transparent_55%)] dark:bg-[radial-gradient(circle_at_top,rgba(14,165,233,.18),transparent_55%),radial-gradient(circle_at_bottom,rgba(79,70,229,.20),transparent_55%)]"
        ></div>

        <div class="mx-auto max-w-6xl px-4 py-16 md:py-20">
          <div class="grid items-center gap-12 md:grid-cols-2">
            <div>
              <p class="text-sm font-semibold text-indigo-600 dark:text-indigo-400">
                About
              </p>
              <h1 class="mt-3 text-4xl font-bold tracking-tight md:text-5xl">
                Built to look great,
                <span
                  class="text-transparent bg-clip-text bg-gradient-to-r from-indigo-600 to-sky-500"
                  >kept intentionally simple</span
                >.
              </h1>
              <p class="mt-4 text-lg leading-relaxed text-slate-600 dark:text-slate-300">
                AstraWeb is a small demo site designed for static hosting and
                learning. The goal: deliver a modern UI without a complicated
                setup.
              </p>

              <div class="mt-8 grid grid-cols-3 gap-3">
                <div
                  class="rounded-2xl border border-slate-200 bg-white/70 p-4 backdrop-blur dark:border-slate-800 dark:bg-slate-950/50"
                >
                  <p class="text-xs text-slate-500 dark:text-slate-400">Focus</p>
                  <p class="mt-1 text-xl font-semibold tracking-tight">UI</p>
                </div>
                <div
                  class="rounded-2xl border border-slate-200 bg-white/70 p-4 backdrop-blur dark:border-slate-800 dark:bg-slate-950/50"
                >
                  <p class="text-xs text-slate-500 dark:text-slate-400">Mode</p>
                  <p class="mt-1 text-xl font-semibold tracking-tight">
                    Static
                  </p>
                </div>
                <div
                  class="rounded-2xl border border-slate-200 bg-white/70 p-4 backdrop-blur dark:border-slate-800 dark:bg-slate-950/50"
                >
                  <p class="text-xs text-slate-500 dark:text-slate-400">
                    Assets
                  </p>
                  <p class="mt-1 text-xl font-semibold tracking-tight">CDN</p>
                </div>
              </div>
            </div>

            <div class="relative">
              <div
                class="overflow-hidden rounded-3xl border border-slate-200 bg-white shadow-lg dark:border-slate-800 dark:bg-slate-900"
              >
                <img
                  class="h-80 w-full object-cover md:h-[420px]"
                  src="https://images.unsplash.com/photo-1521737604893-d14cc237f11d?auto=format&fit=crop&w=1400&q=80"
                  alt="People collaborating at a table with laptops"
                  loading="lazy"
                />
              </div>
            </div>
          </div>
        </div>
      </section>

      <section class="mx-auto max-w-6xl px-4 py-14">
        <div class="grid gap-10 md:grid-cols-2">
          <div>
            <h2 class="text-2xl font-bold tracking-tight">Story</h2>
            <p class="mt-3 text-slate-600 dark:text-slate-300">
              The original pages in this folder were template-driven. This
              redesign turns them into a cohesive, high‑quality static site with
              modern typography, layout, and interactions.
            </p>

            <div class="mt-6 space-y-4">
              <div
                class="rounded-3xl border border-slate-200 bg-white p-6 shadow-sm dark:border-slate-800 dark:bg-slate-950"
              >
                <p class="text-sm font-semibold">Design system</p>
                <p class="mt-2 text-sm leading-relaxed text-slate-600 dark:text-slate-300">
                  A shared header/footer, spacing, and component style across
                  pages.
                </p>
              </div>
              <div
                class="rounded-3xl border border-slate-200 bg-white p-6 shadow-sm dark:border-slate-800 dark:bg-slate-950"
              >
                <p class="text-sm font-semibold">Small interactions</p>
                <p class="mt-2 text-sm leading-relaxed text-slate-600 dark:text-slate-300">
                  Theme toggle, mobile menu, and smooth anchors—no heavy scripts.
                </p>
              </div>
            </div>
          </div>

          <div>
            <h2 class="text-2xl font-bold tracking-tight">Timeline</h2>
            <ol class="mt-5 space-y-4">
              <li
                class="rounded-3xl border border-slate-200 bg-white p-6 shadow-sm dark:border-slate-800 dark:bg-slate-950"
              >
                <p class="text-xs font-semibold text-slate-500 dark:text-slate-400">
                  Step 1
                </p>
                <p class="mt-1 text-sm font-semibold">Rebuild structure</p>
                <p class="mt-2 text-sm text-slate-600 dark:text-slate-300">
                  Replace the old template layout with a consistent page system.
                </p>
              </li>
              <li
                class="rounded-3xl border border-slate-200 bg-white p-6 shadow-sm dark:border-slate-800 dark:bg-slate-950"
              >
                <p class="text-xs font-semibold text-slate-500 dark:text-slate-400">
                  Step 2
                </p>
                <p class="mt-1 text-sm font-semibold">Upgrade the UI</p>
                <p class="mt-2 text-sm text-slate-600 dark:text-slate-300">
                  Modern colors, type scale, spacing, and card components.
                </p>
              </li>
              <li
                class="rounded-3xl border border-slate-200 bg-white p-6 shadow-sm dark:border-slate-800 dark:bg-slate-950"
              >
                <p class="text-xs font-semibold text-slate-500 dark:text-slate-400">
                  Step 3
                </p>
                <p class="mt-1 text-sm font-semibold">Make it usable</p>
                <p class="mt-2 text-sm text-slate-600 dark:text-slate-300">
                  Add links between pages and a clean Services page flow.
                </p>
              </li>
            </ol>
          </div>
        </div>
      </section>

      <section
        class="border-y border-slate-200/70 bg-slate-50 dark:border-slate-800 dark:bg-slate-950/40"
      >
        <div class="mx-auto max-w-6xl px-4 py-14">
          <h2 class="text-2xl font-bold tracking-tight">Toolbox</h2>
          <p class="mt-3 max-w-3xl text-slate-600 dark:text-slate-300">
            This site uses online assets, so it stays lightweight in your
            folder.
          </p>

          <div class="mt-8 flex flex-wrap gap-2">
            <span
              class="rounded-full border border-slate-200 bg-white px-3 py-1 text-xs font-semibold text-slate-700 dark:border-slate-800 dark:bg-slate-950 dark:text-slate-200"
              >Tailwind CSS (CDN)</span
            >
            <span
              class="rounded-full border border-slate-200 bg-white px-3 py-1 text-xs font-semibold text-slate-700 dark:border-slate-800 dark:bg-slate-950 dark:text-slate-200"
              >Google Fonts (Inter)</span
            >
            <span
              class="rounded-full border border-slate-200 bg-white px-3 py-1 text-xs font-semibold text-slate-700 dark:border-slate-800 dark:bg-slate-950 dark:text-slate-200"
              >Unsplash images</span
            >
            <span
              class="rounded-full border border-slate-200 bg-white px-3 py-1 text-xs font-semibold text-slate-700 dark:border-slate-800 dark:bg-slate-950 dark:text-slate-200"
              >Vanilla JS</span
            >
          </div>
        </div>
      </section>

      <section class="mx-auto max-w-6xl px-4 py-16">
        <div
          class="rounded-3xl bg-slate-900 p-10 text-white shadow-lg dark:bg-white dark:text-slate-900"
        >
          <div class="grid items-center gap-8 md:grid-cols-2">
            <div>
              <h2 class="text-2xl font-bold tracking-tight">
                Next: Services & pricing
              </h2>
              <p class="mt-2 text-white/80 dark:text-slate-600">
                Head to the Services page for pricing tiers, process steps, FAQ,
                and a contact section.
              </p>
            </div>
            <div class="flex flex-col gap-3 sm:flex-row md:justify-end">
              <a
                href="services.html"
                class="inline-flex items-center justify-center rounded-xl bg-white px-5 py-3 text-sm font-semibold text-slate-900 shadow-sm hover:bg-slate-100 dark:bg-slate-900 dark:text-white dark:hover:bg-slate-800"
                >Go to Services</a
              >
              <a
                href="index.html"
                class="inline-flex items-center justify-center rounded-xl border border-white/20 bg-white/10 px-5 py-3 text-sm font-semibold text-white shadow-sm hover:bg-white/15 dark:border-slate-300 dark:bg-slate-50 dark:text-slate-900 dark:hover:bg-slate-100"
                >Back home</a
              >
            </div>
          </div>
        </div>
      </section>
    </main>

    <footer class="border-t border-slate-200 py-10 dark:border-slate-800">
      <div
        class="mx-auto flex max-w-6xl flex-col gap-6 px-4 md:flex-row md:items-center md:justify-between"
      >
        <div class="flex items-center gap-3">
          <span
            class="grid h-10 w-10 place-items-center rounded-2xl bg-gradient-to-br from-indigo-600 to-sky-500 text-sm font-extrabold text-white"
            >AW</span
          >
          <div>
            <p class="text-sm font-semibold">AstraWeb</p>
            <p class="text-xs text-slate-500 dark:text-slate-400">About</p>
          </div>
        </div>

        <div class="flex flex-wrap gap-x-6 gap-y-3 text-sm">
          <a
            class="text-slate-600 hover:text-slate-900 dark:text-slate-300 dark:hover:text-white"
            href="index.html"
            >Home</a
          >
          <a
            class="text-slate-600 hover:text-slate-900 dark:text-slate-300 dark:hover:text-white"
            href="about.html"
            >About</a
          >
          <a
            class="text-slate-600 hover:text-slate-900 dark:text-slate-300 dark:hover:text-white"
            href="services.html"
            >Services</a
          >
        </div>
      </div>
    </footer>

    <script>
      const toggle = document.getElementById("themeToggle");
      toggle?.addEventListener("click", () => {
        const isDark = document.documentElement.classList.toggle("dark");
        localStorage.setItem("theme", isDark ? "dark" : "light");
      });
    </script>
  </body>
</html>


File di atas disimpan dengan nama about.html pada direktori yang sama pada folder python yang anda simpan sebelumnya.

##### Langkah Uji Coba
1. Pastikan file `Static-Webserver.py`, `index.html`, `about.html`, dan `services.html` berada dalam 1 folder/direktori yang sama.
2. Jalankan server dengan perintah `python Static-Webserver.py`.
3. Buka browser dan akses `http://127.0.0.1:8080/` untuk memuat halaman `index.html`.
4. Coba akses halaman lain melalui navigasi/URL: `http://127.0.0.1:8080/about.html` dan `http://127.0.0.1:8080/services.html`.
5. Uji halaman yang tidak ada, misalnya `http://127.0.0.1:8080/abc.html`, untuk melihat respon `404 Not Found`.
6. Amati output log pada terminal (alamat klien, request line, timestamp, dan status code).

##### Analisa Hasil
- Saat browser mengakses server, program menerima request melalui socket, lalu membaca `request_line` (mis. `GET / HTTP/1.1`).
- Fungsi `route_request()` memetakan path `/` menjadi `index.html`, sedangkan path lain akan menghapus `/` di depan (contoh `/about.html` menjadi `about.html`).
- Jika file ada, server mengirim respon `HTTP/1.1 200 OK` dan header `Content-Type` sesuai ekstensi file (mis. `.html` -> `text/html`).
- Jika file tidak ditemukan, server mengirim halaman sederhana dengan status `HTTP/1.1 404 Not Found`.
- Jika terjadi error saat membaca/mengirim file, server mengirim status `HTTP/1.1 500 Internal Server Error`.
- Setiap request dicatat oleh `log_request()` sehingga kita bisa melihat request yang masuk dan status responnya.
- Website yang dihasilkan bersifat statis (hanya file HTML/CSS/JS/gambar), sehingga server hanya bertugas “mengambil file lalu mengirimkannya”, tanpa proses backend/framework.


#### 1.5 Kesimpulan
Webserver Python sederhana berbasis socket dapat digunakan untuk melayani website statis multi‑page (Home, About, Services) dengan cara memetakan URL ke file pada direktori yang sama. Server mampu memberikan respon dasar HTTP dengan status `200`, `404`, dan `500`, serta mengatur `Content-Type` sesuai jenis file. Dari uji coba, halaman dapat diakses melalui browser dan setiap permintaan tercatat pada log, sehingga memudahkan proses analisis request/response. Keterbatasannya adalah fitur HTTP masih minimal (tanpa HTTPS, routing kompleks, atau concurrency tingkat lanjut) sehingga cocok untuk pembelajaran dasar konsep web server statis.